# 3D fault modelling

In this example, we simulate basin formation in the depression induced by a combined normal and strike-slip fault system. 

<div align="center">
    <img src="images/settings.png" alt="fault system" width="500" height="200"/>
</div>

We investigate the impact of erodibility coefficient on preservation of the basin over 100k years. 

Each run starts with an inclined topography ($<0.5/100$) over which a constant precipitation rate (1 m/a) is applied. After 2000 years the 3D displacements combining the normal and strike-slip components (kindly provided by [Guillaume Duclaux](http://www.uib.no/en/persons/Guillaume.Duclaux)) are applied. The model applies 5 times the same displacements up to the simulation end time (100k years). 

## Initial settings

For this model, we use the *stream power law sediment transport model* which scale the incision rate $E$ as a power function of surface water discharge $A$ and slope $S=\nabla z$:

$$ E = \kappa A^m (\nabla z)^n$$

where $\kappa$ is the erodibility coefficient dependent on lithology and mean precipitation rate, channel width, flood frequency, channel hydraulics.

The values given to these parameters ($\kappa$, $m$, $n$) need to be set in the **XmL** input file. To evaluate the role of the erodibility coefficient we keep the values of $m$ and $n$ constant for each of our run but we will change the coefficient of erodibility between each run:
+ $\kappa = 2.5 e^{-6}$ /a
+ $\kappa = 5.0 e^{-6}$ /a
+ $\kappa = 1.0 e^{-5}$ /a

In this model, we need the top surface (TIN) is reevaluate every 500 years to make sure the Delaunay triangulation distortion induced by the horizontal displacements is minimal. If this is not the case the surface is automatically remeshed to:
+ ensure homogeneous nodes distribution when sparse areas exist
+ limit nodes concentration in node dense areas

## Starting pyBadlands

First we initialise the model and set the path to the **XmL** input file.

You can edit the **XmL** configuration file at [/edit/volume/Examples/strikeslip/strikeslip.xml](/edit/volume/Examples/strikeslip/strikeslip.xml).

To view the complete **XmL** options you can follow this link to github page: [complete.xml](https://github.com/badlands-model/pyBadlands/blob/master/Examples/input_complete.xml).

In [ ]:
from pyBadlands.model import Model as badlandsModel
# If you'd like to run this across an MPI cluster, instead use
#from pyBadlands.remote import RemoteModel as badlandsModel

# initialise model
model = badlandsModel()
model.load_xml('strike-slip.xml')

## Running pyBadlands

We can run the model for a given period. The end time in the **XmL** input file is set to _100k years_ but you might want to run the model for a coupled of iterations and check the output before running the model for the entire simulation time. This is done by putting the time in the **run_to_time** function. 

Here we go for the full time directly... it should take between 5 and 15 minutes depending of the chosen coefficient value on a single processor if you keep the other parameters unchanged. 

In [ ]:
model.run_to_time(10000)

## Visualise output in Notebook

For quick visualisation of the output at a given time step you can use 2 funtions:
+ visSurf
+ plotSurf

The functions are based on [plot.ly](https://plot.ly) librairy and will give you a simple interactive view of the model outputs.

In [ ]:
from pyBadlands import visSurf

view = visSurf.visSurf(folder=model.input.outDir, ncpus=model.ncpus(), dx=model.recGrid.resEdges, timestep=1)
view.plotSurf(width = 800, height = 600, zmin = -8000, zmax = 1000, color = 'Earth', reverse=False,
              dataV = 'z', subsample = 2)

view = visSurf.visSurf(folder=model.input.outDir, ncpus=model.ncpus(), dx=model.recGrid.resEdges, timestep=1,
                       crange=[-400,200])
view.plotSurf(width = 800, height = 600, zmin = -1000, zmax = 1000, color = 'RdBu', reverse=False,
              dataV = 'c', subsample = 2)

# Visualise in Paraview

For an in-depth visualisation of the model output it is recommended to install on your local machine a visualisation package such as [Paraview](http://www.paraview.org) or [Visit](https://wci.llnl.gov/simulation/computer-codes/visit/).

**Badlands** outputs are created in the directory you've specified in your **XmL** input file. In cases where you have ran several instances of the same input you will end up with multiple directories with the same name followed with the simulation number.

The folder contains xdmf series files:
+ **tin.series.xdmf** which will display the time series of surface evolution.
+ **flow.series.xdmf** which will display the time series of drainage network evolution.

### Elevation change

| Time step 25| Time step 160 | 
| :-------------: |:-------------:|
| <img src="images/step10.png" alt="Step 10" width="400" height="300" border="10" />| <img src="images/step25.png" alt="Step 25" width="400" height="300" border="10" /> | 

| Time step 250| Time step 400| 
| :-------------: |:-------------:|
| <img src="images/step38.png" alt="Step 38" width="400" height="300" border="10" /> | <img src="images/step49.png" alt="Step 49" width="400" height="300" border="10" />|

### Comparisons between models

| Different $\kappa$ values| 
|:-------------:|
|<img src="images/step50.png" alt="Step 50" width="500" height="300"/>|

In the next notebooks we will use the [Badlands companion](https://github.com/badlands-model/pyBadlands-Companion)  python classes to look at some of the available post-processing functions to visualise the volume of sediment deposited through time for the different erodibility coefficients.